### Task description

So here is what we could do: A code needs to be written that takes in the quants and our metadata (so basically is it understand our groups). If the quant from mzmine is non-zero in two of three triplicates then leave it but if it is non-zero in only one and zero in two replicates, then set them to zero in the output quant. 



### Base code

In [4]:
# Fazer o download direto das tabelas de dados e metadados
# modifique o valor de taskid para outro job
import io
import requests
import pandas as pd

taskid = '898c8b490165455bbeb798e89400586d'

base_url = 'http://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task='

url_to_metadata = f'{base_url}{taskid}&block=main&file=metadata_table/'
url_to_features = f'{base_url}{taskid}&block=main&file=quantification_table/'

meta = pd.read_csv(io.StringIO(requests.get(url_to_metadata).text), sep='\t')
feat = pd.read_csv(io.StringIO(requests.get(url_to_features).text))

In [5]:
# Cada arquivo na coluna filename representa uma amostra
# as outras colunas são as informações das amostras
meta.head()

,filename,ATTRIBUTE_strain,ATTRIBUTE_mutant,ATTRIBUTE_media,ATTRIBUTE_Trimethoprim,ATTRIBUTE_extraction,ATTRIBUTE_triplicate
0,BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML,H111,H111_cepI,SCFM2,noTMP,EA,A
1,BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML,H111,H111_cepI,SCFM2,noTMP,EA,A
2,BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML,H111,H111_cepI,SCFM2,noTMP,EA,A
3,BcJ2315_cciI_noTMP_SPE_1_P1-C-1_01_12559.mzXML,J2315,J2315_cciI,LB,noTMP,SPE,AB
4,BcJ2315_cciI_noTMP_SPE_2_P1-C-2_01_12560.mzXML,J2315,J2315_cciI,LB,noTMP,SPE,AB


In [6]:
# Cada amostra tem uma coluna, com a quantidade detectada
# de cada metabóólito (m/z e retention time)
feat.head()

,row ID,row m/z,row retention time,BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML Peak area,BcH111_cepI_noTMP_SPE_2_P1-C-2_01_12970.mzXML Peak area,BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML Peak area,BcH111_cepI_noTMP_SPE_1_P1-C-1_01_12969.mzXML Peak area,BcH111_rpfA_noTMP_EA_1_P1-C-7_01_12978.mzXML Peak area,BcH111_rpfA_noTMP_EA_2_P1-C-8_01_12979.mzXML Peak area,BcH111_cepI_noTMP_SPE_3_P1-C-3_01_12971.mzXML Peak area,BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML Peak area,BcJ2315_cepI_noTMP_EA_3_P1-C-9_01_12857.mzXML Peak area,BcJ2315_cciI_noTMP_EA_1_SCFM2_P1-B-2_01_12779.mzXML Peak area,BcJ2315_cciI_noTMP_EA_2_SCFM2_P1-B-3_01_12780.mzXML Peak area,BcJ2315_cciI_noTMP_SPE_1_SCFM2_P1-B-8_01_12788.mzXML Peak area,BcJ2315_cepI_noTMP_EA_1_P1-C-7_01_12855.mzXML Peak area,BcJ2315_cciI_noTMP_SPE_2_SCFM2_P1-B-9_01_12789.mzXML Peak area,BcJ2315_cciI_noTMP_EA_3_SCFM2_P1-B-4_01_12781.mzXML Peak area,BcJ2315_cepI_noTMP_SPE_2_P1-D-5_01_12865.mzXML Peak area,BcJ2315_cepI_noTMP_EA_2_P1-C-8_01_12856.mzXML Peak area,BcH111_rpfA_noTMP_EA_3_P1-C-9_01_12980.mzXML Peak area,BcJ2315_double_mut_noTMP_EA_1_P1-A-1_01_12817.mzXML Peak area,BcJ2315_cepI_noTMP_SPE_1_P1-D-4_01_12864.mzXML Peak area,BcJ2315_cepI_noTMP_SPE_3_P1-D-6_01_12866.mzXML Peak area,BcJ2315_double_mut_noTMP_EA_2_P1-A-2_01_12818.mzXML Peak area,BcJ2315_double_mut_noTMP_EA_3_P1-A-3_01_12819.mzXML Peak area,BcJ2315_cciI_noTMP_SPE_3_SCFM2_P1-C-1_01_12790.mzXML Peak area,BcH111_rpfA_noTMP_SPE_1_P1-D-4_01_12987.mzXML Peak area,BcJ2315_rpfA_noTMP_EA_3_SCFM2_P1-E-1_01_12800.mzXML Peak area,BcJ2315_rpfA_noTMP_SPE_1_SCFM2_P1-E-5_01_12808.mzXML Peak area,BcJ2315_rpfA_noTMP_EA_2_SCFM2_P1-D-9_01_12799.mzXML Peak area,BcJ2315_rpfA_noTMP_EA_1_SCFM2_P1-D-8_01_12798.mzXML Peak area,BcJ2315_double_mut_noTMP_SPE_1_P1-A-7_01_12826.mzXML Peak area,BcJ2315_double_mut_noTMP_SPE_2_P1-A-8_01_12829.mzXML Peak area,BcJ2315_triple_mut_noTMP_EA_2_P1-B-5_01_12838.mzXML Peak area,BcK562_cciI_noTMP_EA_2_P1-B-5_01_12907.mzXML Peak area,BcJ2315_triple_mut_noTMP_EA_1_P1-B-4_01_12837.mzXML Peak area,BcJ2315_rpfA_noTMP_SPE_3_SCFM2_P1-E-7_01_12810.mzXML Peak area,BcJ2315_triple_mut_noTMP_EA_3_P1-B-6_01_12839.mzXML Peak area,BcH111_rpfA_noTMP_SPE_2_P1-D-5_01_12988.mzXML Peak area,...,BcK562_WT_TMP+_EA_2_P1-D-2_01_12654.mzXML Peak area,BcK562_cciI_noTMP_EA_2_P1-A-1_01_12668.mzXML Peak area,BcJ2315_cciI_TMP+_SPE_3_P1-C-6_01_12564.mzXML Peak area,BcK562_WT_noTMP_SPE_2_P1-D-5_01_12660.mzXML Peak area,BcJ2315_cciI_TMP+_SPE_2_P1-C-5_01_12563.mzXML Peak area,BcJ2315_rpfF_TMP+_SPE_1_P1-F-5_01_12602.mzXML Peak area,BcK562_WT_TMP+_EA_1_P1-D-1_01_12653.mzXML Peak area,BcJ2315_cepI_TMP+_SPE_1_P1-D-7_01_12584.mzXML Peak area,BcK562_cepI_noTMP_EA_3_P1-B-5_01_12688.mzXML Peak area,BcK562_cciI_TMP+_EA_2_P1-A-4_01_12671.mzXML Peak area,BcK562_cciI_TMP+_EA_3_P1-A-5_01_12672.mzXML Peak area,BcK562_WT_noTMP_SPE_1_P1-D-4_01_12659.mzXML Peak area,BcK562_WT_noTMP_SPE_3_P1-D-6_01_12661.mzXML Peak area,BcK562_cciI_noTMP_SPE_2_P1-A-7_01_12677.mzXML Peak area,BcK562_cciI_TMP+_EA_1_P1-A-3_01_12670.mzXML Peak area,BcK562_cciI_noTMP_SPE_3_P1-A-8_01_12678.mzXML Peak area,BcK562_cepI_TMP+_EA_2_P1-B-7_01_12690.mzXML Peak area,BcJ2315_rpfF_TMP+_SPE_2_P1-F-6_01_12603.mzXML Peak area,BcK562_cciI_noTMP_SPE_1_P1-A-6_01_12676.mzXML Peak area,BcK562_cepI_TMP+_EA_1_P1-B-6_01_12689.mzXML Peak area,BcK562_WT_TMP+_SPE_2_P1-D-8_01_12663.mzXML Peak area,BcK562_WT_TMP+_SPE_3_P1-D-9_01_12664.mzXML Peak area,BcK562_cepI_noTMP_SPE_1_P1-B-9_01_12695.mzXML Peak area,BcK562_cepI_TMP+_EA_3_P1-B-8_01_12691.mzXML Peak area,LB_EA_3_P1-A-3_01_12997.mzXML Peak area,BcK562_cepI_noTMP_SPE_3_P1-C-2_01_12697.mzXML Peak area,BcK562_cciI_TMP+_SPE_2_P1-B-1_01_12680.mzXML Peak area,BcK562_cepI_noTMP_SPE_2_P1-C-1_01_12696.mzXML Peak area,LB_EA_2_P1-A-2_01_12996.mzXML Peak area,BcK562_WT_TMP+_SPE_1_P1-D-7_01_12662.mzXML Peak area,BcK562_cciI_TMP+_SPE_3_P1-B-2_01_12681.mzXML Peak area,LB_EA_1_P1-A-1_01_12995.mzXML Peak area,BcK562_cepI_TMP+_SPE_3_P1-C-5_01_12700.mzXML Peak area,BcK562_cciI_

In [10]:
# Coluna informando que amostras representam repetições
meta['ATTRIBUTE_triplicate'].value_counts()

BS    4
CW    4
CX    4
CP    3
Y     3
     ..
BX    2
BR    2
BV    2
J     2
BO    2
Name: ATTRIBUTE_triplicate, Length: 100, dtype: int64

In [11]:
# Quais amostras representam a replicata 'A'
meta[meta['ATTRIBUTE_triplicate']=='A']

,filename,ATTRIBUTE_strain,ATTRIBUTE_mutant,ATTRIBUTE_media,ATTRIBUTE_Trimethoprim,ATTRIBUTE_extraction,ATTRIBUTE_triplicate
0,BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML,H111,H111_cepI,SCFM2,noTMP,EA,A
1,BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML,H111,H111_cepI,SCFM2,noTMP,EA,A
2,BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML,H111,H111_cepI,SCFM2,noTMP,EA,A


In [12]:
# selecionando o nome das amostras
meta.loc[meta['ATTRIBUTE_triplicate']=='A', 'filename']

0    BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML
1    BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML
2    BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML
Name: filename, dtype: object

In [16]:
# Acessar nomes das colunas
feat.columns

Index(['row ID', 'row m/z', 'row retention time',
       'BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML Peak area',
       'BcH111_cepI_noTMP_SPE_2_P1-C-2_01_12970.mzXML Peak area',
       'BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML Peak area',
       'BcH111_cepI_noTMP_SPE_1_P1-C-1_01_12969.mzXML Peak area',
       'BcH111_rpfA_noTMP_EA_1_P1-C-7_01_12978.mzXML Peak area',
       'BcH111_rpfA_noTMP_EA_2_P1-C-8_01_12979.mzXML Peak area',
       'BcH111_cepI_noTMP_SPE_3_P1-C-3_01_12971.mzXML Peak area',
       ...
       'BcK562_cciI_TMP+_SPE_3_P1-B-2_01_12681.mzXML Peak area',
       'LB_EA_1_P1-A-1_01_12995.mzXML Peak area',
       'BcK562_cepI_TMP+_SPE_3_P1-C-5_01_12700.mzXML Peak area',
       'BcK562_cciI_TMP+_SPE_1_P1-A-9_01_12679.mzXML Peak area',
       'BcK562_cepI_TMP+_SPE_2_P1-C-4_01_12699.mzXML Peak area',
       'LB_SPE_3_P1-A-6_01_13000.mzXML Peak area',
       'LB_SPE_2_P1-A-5_01_12999.mzXML Peak area',
       'BcK562_cepI_TMP+_SPE_1_P1-C-3_01_12698.mzXML Peak area',
    

In [15]:
# verificar se o nome das amostras de interesse está nas colunas
selected_files = meta.loc[meta['ATTRIBUTE_triplicate']=='A', 'filename']
feat.columns.isin(selected_files) 

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [17]:
# modificar o nome das colunas para parear o nome dos arqivos desejados
feat.columns.str.replace(' Peak area', '')

Index(['row ID', 'row m/z', 'row retention time',
       'BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML',
       'BcH111_cepI_noTMP_SPE_2_P1-C-2_01_12970.mzXML',
       'BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML',
       'BcH111_cepI_noTMP_SPE_1_P1-C-1_01_12969.mzXML',
       'BcH111_rpfA_noTMP_EA_1_P1-C-7_01_12978.mzXML',
       'BcH111_rpfA_noTMP_EA_2_P1-C-8_01_12979.mzXML',
       'BcH111_cepI_noTMP_SPE_3_P1-C-3_01_12971.mzXML',
       ...
       'BcK562_cciI_TMP+_SPE_3_P1-B-2_01_12681.mzXML',
       'LB_EA_1_P1-A-1_01_12995.mzXML',
       'BcK562_cepI_TMP+_SPE_3_P1-C-5_01_12700.mzXML',
       'BcK562_cciI_TMP+_SPE_1_P1-A-9_01_12679.mzXML',
       'BcK562_cepI_TMP+_SPE_2_P1-C-4_01_12699.mzXML',
       'LB_SPE_3_P1-A-6_01_13000.mzXML', 'LB_SPE_2_P1-A-5_01_12999.mzXML',
       'BcK562_cepI_TMP+_SPE_1_P1-C-3_01_12698.mzXML',
       'LB_SPE_1_P1-A-4_01_12998.mzXML', 'Unnamed: 299'],
      dtype='object', length=300)

In [18]:
# Substituir o nome das colunas e testar novamente
feat.columns = feat.columns.str.replace(' Peak area', '')
feat.columns.isin(selected_files) # veja como obteremos o valor True para os arquivos desejados

array([False, False, False,  True, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [19]:
# Podemos contar os valores verdadeiros
feat.columns.isin(selected_files).sum()

3

In [20]:
# selecionar a quantidade dos metabóólitos nas colunas desejadas
feat[selected_files].head() # exibir apenas o cabeçalho, utilize .shape para saber o formato da tabela

,BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML,BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML,BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML
0,4.223396e+08,4.300826e+08,3.396926e+08
1,3.588251e+08,5.442658e+08,4.416860e+08
2,9.287189e+07,7.887541e+07,1.376501e+08
3,3.791783e+07,7.443308e+07,6.534912e+07
4,6.200470e+07,5.834584e+07,5.523902e+07


In [21]:
# exemplo de como saber se UM metabóólito está presente em pelo menos 2 amostras
feat.loc[0, selected_files]

BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML    4.223396e+08
BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML    4.300826e+08
BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML    3.396926e+08
Name: 0, dtype: float64

In [22]:
# exemplo de como saber se UM metabóólito está presente em pelo menos 2 amostras
feat.loc[0, selected_files] > 0

BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML    True
BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML    True
BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML    True
Name: 0, dtype: bool

In [23]:
(feat.loc[0, selected_files] > 0).sum()

3

In [25]:
(feat.loc[0, selected_files] > 0).sum() > 1

True

In [27]:
# exemplo de como saber se TODOS metabóólitos estão presente em pelo menos 2 amostras
# exemplo 1
indices = []
for i in feat[selected_files].head().index:
  if (feat.loc[i, selected_files] > 0).sum() > 1:
  # veja como estamo usando i como indice em cada iteracao do loop
    indices.append(i)

print('Indices encontrados:', indices)

Indices encontrados: [0, 1, 2, 3, 4]


In [31]:
# exemplo de como saber se TODOS metabóólitos estão presente em pelo menos 2 amostras
# exemplo 2
feat[selected_files].head().apply(lambda a: len(a), axis=1)  # navegar em cada linha, cada linha éé guardada na variável a 


0    3
1    3
2    3
3    3
4    3
dtype: int64

In [33]:
# exemplo de como saber se TODOS metabóólitos estão presente em pelo menos 2 amostras
# exemplo 2
feat[selected_files].head().apply(lambda a: (a>0).sum()>1, axis=1)

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [34]:
# executar para todas
feat[selected_files].apply(lambda a: (a>0).sum()>1, axis=1)

0         True
1         True
2         True
3         True
4         True
         ...  
26881    False
26882    False
26883    False
26884    False
26885    False
Length: 26886, dtype: bool

In [35]:
# saber quantas atendem ao critério
feat[selected_files].apply(lambda a: (a>0).sum()>1, axis=1).sum()

4467

## Descrição da tarefa:

Escrever uma função, que pega do usuário a string _taskid_ recupera a tabela de dados e metadados e retorna a resposta no seguinte formato:

nomes das rééplicas nas colunas e identificação dos metabóólitos nas linhas:

row ID | A | AB | ...
--|--|-- | --
1 | True | True |
2 | False | True |
3 | False | False |
...


In [28]:
import io
import requests
import pandas as pd


def taskid_table(taskid, mode):
    """
    Takes strings parameters (taskid and mode) and generate a table that features the triplicates groups and metabolites.
    If the quant from mzmine is non-zero in two of three triplicates then its value is set to True but if it is
    non-zero in just one and zero in two replicates its value will be False.
    This table can be filtered to only contain the metabolites that are non-zero from at least two replicates in all
    triplicates groups. It means that all metabolites that have any False value will be dropped. Or you can
    generate the table without the filter to have all values.

    Parameters:
    taskid (str): taskid provided by user

    mode (str): With this parameter you select which table will be generated. Options are:
        'all_with_filter' : you produce a table with all triplicate groups and only metabolites that the quant from
        mzmine is non-zero in two of three triplicates for all replicates groups.
        any triplicate group name : If you only want a specific group of triplicates in the table, you can use the name
        of this group as this parameter like for example 'A' and the generated table will contain this group and the
        metabolites which are non-zero in two of the three triplicates in this group.
        'all_no_filter' : a table with all metabolites and triplicates will be generated without filter,
        having all values.
    """

    base_url = 'http://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task='
    url_to_metadata = f'{base_url}{taskid}&block=main&file=metadata_table/'
    url_to_features = f'{base_url}{taskid}&block=main&file=quantification_table/'
    meta = pd.read_csv(io.StringIO(requests.get(url_to_metadata).text), sep='\t')
    feat = pd.read_csv(io.StringIO(requests.get(url_to_features).text))
    feat.columns = feat.columns.str.replace(' Peak area', '')
    list_triplicates = list(meta['ATTRIBUTE_triplicate'].unique()) #pegando a lista dos grupos de replicatas e
                                                                    #tirando repeticoes
    newdfdict = {}
    print(list_triplicates)
    print(''*30)
    
    
    for key in list_triplicates:
        selected_files = meta.loc[meta['ATTRIBUTE_triplicate'] == key, 'filename'] # selecionando todas as amostras
                                                                                    # por grupos de triplicatas
        print(f'Os arquivos selecionados para o grupo {key}:')
        print(selected_files)
        print('-'*25)
        print(f'O df com os selected files eh:')
        print(feat[selected_files].head() > 0)
        
        
        newdfdict[key] = feat[selected_files].head().apply(lambda a: (a > 0).sum() > 1, axis=1)
        print(''*25)
        
        print(f'E os valores adicionados para {key} no dicionario sao: {list(newdfdict[key])}')
        print('' * 25)

    df = pd.DataFrame.from_dict(newdfdict)
    print(df)
    df['row ID'] = feat[['row ID']]
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df1 = df[cols]
    print(df)
    print('-' * 30)
    print('df com as colunas arrumadas:)
    print(df1)
    if mode == 'all_with_filter':
        df2 = df1[(df1 > 0).all(axis=1)]
        print('df com filtro aplicado:')
        print(df2)
        df2.to_csv('allgroups.tsv', sep='\t', index=False)
    elif mode in list_triplicates:
        df2 = df1[['row ID', mode]][df1[mode] > 0]
        df2.to_csv(f'only_{mode}.tsv', sep='\t', index=False, header=True)
    elif mode == 'all_no_filter':
        df1.to_csv('all_features_no_filter.tsv', sep='\t', index=False)
    



taskid_table('898c8b490165455bbeb798e89400586d', 'all_with_filter')

['A', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'B', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'C', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CU', 'CV', 'CW', 'CX', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

Os arquivos selecionados para o grupo A:
0    BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML
1    BcH111_cepI_noTMP_EA_2_P1-B-5_01_12961.mzXML
2    BcH111_cepI_noTMP_EA_3_P1-B-6_01_12962.mzXML
Name: filename, dtype: object
-------------------------
O df com os selected files eh:
   BcH111_cepI_noTMP_EA_1_P1-B-4_01_12960.mzXML  \
0                                          True   
1                                          T

   BcJ2315_cepI_noTMP_EA_1_P1-C-7_01_12572.mzXML  \
0                                          False   
1                                          False   
2                                           True   
3                                           True   
4                                           True   

   BcJ2315_cepI_noTMP_EA_2_P1-C-8_01_12573.mzXML  \
0                                          False   
1                                          False   
2                                           True   
3                                           True   
4                                           True   

   BcJ2315_cepI_noTMP_EA_3_P1-C-9_01_12574.mzXML  
0                                          False  
1                                          False  
2                                           True  
3                                           True  
4                                           True  

E os valores adicionados para AH no dicionario sao: [False, False, 


E os valores adicionados para AO no dicionario sao: [False, False, True, True, True]

Os arquivos selecionados para o grupo AP:
45    BcJ2315_double_mut_noTMP_EA_1_P1-A-1_01_12612....
46    BcJ2315_double_mut_noTMP_EA_2_P1-A-2_01_12613....
47    BcJ2315_double_mut_noTMP_EA_3_P1-A-3_01_12614....
Name: filename, dtype: object
-------------------------
O df com os selected files eh:
   BcJ2315_double_mut_noTMP_EA_1_P1-A-1_01_12612.mzXML  \
0                                              False     
1                                               True     
2                                               True     
3                                               True     
4                                               True     

   BcJ2315_double_mut_noTMP_EA_2_P1-A-2_01_12613.mzXML  \
0                                              False     
1                                              False     
2                                               True     
3                                  


E os valores adicionados para AU no dicionario sao: [False, False, True, True, True]

Os arquivos selecionados para o grupo AV:
63    BcJ2315_double_mut_TMP+_SPE_1_P1-B-1_01_12624....
64    BcJ2315_double_mut_TMP+_SPE_2_P1-B-2_01_12625....
65    BcJ2315_double_mut_TMP+_SPE_3_P1-B-3_01_12626....
Name: filename, dtype: object
-------------------------
O df com os selected files eh:
   BcJ2315_double_mut_TMP+_SPE_1_P1-B-1_01_12624.mzXML  \
0                                              False     
1                                              False     
2                                              False     
3                                               True     
4                                               True     

   BcJ2315_double_mut_TMP+_SPE_2_P1-B-2_01_12625.mzXML  \
0                                              False     
1                                              False     
2                                               True     
3                                  

O df com os selected files eh:
   BcJ2315_rpfA_TMP+_SPE_1_SCFM2_P1-E-8_01_12811.mzXML  \
0                                              False     
1                                              False     
2                                               True     
3                                               True     
4                                               True     

   BcJ2315_rpfA_TMP+_SPE_2_SCFM2_P1-E-9_01_12812.mzXML  \
0                                              False     
1                                              False     
2                                               True     
3                                               True     
4                                               True     

   BcJ2315_rpfA_TMP+_SPE_3_SCFM2_P1-F-9_01_12813.mzXML  
0                                              False    
1                                              False    
2                                               True    
3                                          

...


E os valores adicionados para CE no dicionario sao: [False, False, False, True, True]

Os arquivos selecionados para o grupo CF:
170    BcK562_cepI_noTMP_EA_1_P1-C-7_01_12924.mzXML
171    BcK562_cepI_noTMP_EA_2_P1-C-8_01_12925.mzXML
172    BcK562_cepI_noTMP_EA_3_P1-C-9_01_12926.mzXML
Name: filename, dtype: object
-------------------------
O df com os selected files eh:
   BcK562_cepI_noTMP_EA_1_P1-C-7_01_12924.mzXML  \
0                                         False   
1                                         False   
2                                         False   
3                                          True   
4                                          True   

   BcK562_cepI_noTMP_EA_2_P1-C-8_01_12925.mzXML  \
0                                         False   
1                                          True   
2                                         False   
3                                          True   
4                                          True   

   BcK562_cep


E os valores adicionados para CL no dicionario sao: [False, True, False, True, True]

Os arquivos selecionados para o grupo CM:
191    BcK562_WT_noTMP_EA_1_P1-A-1_01_12888.mzXML
192    BcK562_WT_noTMP_EA_2_P1-A-2_01_12889.mzXML
193    BcK562_WT_noTMP_EA_3_P1-A-3_01_12890.mzXML
Name: filename, dtype: object
-------------------------
O df com os selected files eh:
   BcK562_WT_noTMP_EA_1_P1-A-1_01_12888.mzXML  \
0                                       False   
1                                        True   
2                                       False   
3                                        True   
4                                        True   

   BcK562_WT_noTMP_EA_2_P1-A-2_01_12889.mzXML  \
0                                       False   
1                                        True   
2                                       False   
3                                        True   
4                                        True   

   BcK562_WT_noTMP_EA_3_P1-A-3_01_12890.mzXM

   LB_SPE_1_P1-A-4_01_12998.mzXML  LB_SPE_2_P1-A-5_01_12999.mzXML  \
0                           False                           False   
1                           False                           False   
2                           False                           False   
3                            True                            True   
4                            True                            True   

   LB_SPE_3_P1-A-6_01_13000.mzXML  
0                           False  
1                           False  
2                           False  
3                            True  
4                            True  

E os valores adicionados para CV no dicionario sao: [False, False, False, True, True]

Os arquivos selecionados para o grupo CW:
221    SCFM2_EA_1_P1-A-7_01_13004.mzXML
222    SCFM2_EA_2_P1-A-8_01_13005.mzXML
223    SCFM2_EA_3_P1-A-9_01_13006.mzXML
224    SCFM2_EA_4_P1-B-1_01_13007.mzXML
Name: filename, dtype: object
-------------------------
O df com os selected fi

   BcH111_cepI_TMP+_SPE_1_P1-C-4_01_12972.mzXML  \
0                                          True   
1                                         False   
2                                         False   
3                                          True   
4                                          True   

   BcH111_cepI_TMP+_SPE_2_P1-C-5_01_12973.mzXML  \
0                                          True   
1                                         False   
2                                         False   
3                                          True   
4                                          True   

   BcH111_cepI_TMP+_SPE_3_P1-C-6_01_12974.mzXML  
0                                          True  
1                                         False  
2                                         False  
3                                          True  
4                                          True  

E os valores adicionados para H no dicionario sao: [True, False, False, True, True]



   BcH111_WT_TMP+_SPE_1_P1-E-1_01_12716.mzXML  \
0                                       False   
1                                       False   
2                                       False   
3                                        True   
4                                        True   

   BcH111_WT_TMP+_SPE_2_P1-E-2_01_12717.mzXML  \
0                                       False   
1                                       False   
2                                       False   
3                                        True   
4                                        True   

   BcH111_WT_TMP+_SPE_3_P1-E-3_01_12718.mzXML  
0                                        True  
1                                       False  
2                                       False  
3                                        True  
4                                        True  

E os valores adicionados para X no dicionario sao: [False, False, False, True, True]

Os arquivos selecionados para o gru

as chaves do dicionario seram os grupos de triplicatas, e o valores dessas chaves vao ser uma lista [True, False, etc] com os valores para cada metabolito.
a funcao lambda esta pedindo para somar todos os valores das amostras (cada valor eh a variavel a) por metabolito

Primeiro ele verifica se a > 0, se for, ele retorna True para o valor, e se nao for ele retorna False
Ele faz isso com todas as amostras de determinado grupo de triplicata - dando uma lista de True False True por exemplo
Depois ele soma esses valores (True sempre assume valor = 1 e False =0) por metabolito. 
Se essa soma der maior que 1, isso significa que no minimo 2 amostras possuem valor True, isto eh, 
valor maior que 0 para tal metabolito.
Entao ele vai adicionar na chave A por exemplo, que o valor para ela eh True, porque ela tem no minimo 2 amostras
com valores maiores que 0.
Caso a soma seja menor que 1, o valor que eh dado pra determinado metabolito vai ser False